In [1]:
import random
import simpy
import pandas as pd
from simpy.util import start_delayed

In [2]:
class Building(object):
    def __init__(self, env, building):
        self.setAddress(building)
#         self.setResident(building)
        try:
            self.setOccupancy(building)
        except AttributeError:
            print('Invalid occupancy type provided: {0}.'.format(building['occupancy']))
        self.setMonthlyCost(building)
        self.setRooms(building)
        self.setFloors(building)
        self.setAge(building)
        self.setValue(building)
        self.setDamage(building)  
        self.setInspection(env, building)
    def setAddress(self, building):
        self.address = building['address']
    def setOccupancy(self, building):
        self.occupancy = building['occupancy']
    def setMonthlyCost(self, building):
        self.cost = building['cost']
    def setRooms(self, building):
        self.rooms = building['rooms']
    def setFloors(self, building):
        self.floors = building['floors']
    def setAge(self, building):
        self.age = building['age']
    def setValue(self, building):
        self.value = building['value']
    def setDamage(self, building):
        self.damage = building['damage']
    def setInspection(self, env, building):
        self.inspection = env.event()
        self.inspect_start = None
        self.inspect_stop = None


In [3]:
class Residence(Building):
    #Can verify that occupancy types only relate to residences
    def setOccupancy(self, building):
        if building['occupancy'] in ('house', 'apartment', 'condo'):
            self.occupancy = building['occupancy']
        else:
            raise AttributeError(building['occupancy'])

In [4]:
class Household(object):
    def __init__(self, env, household):
        self.name = household['name']
        self.income = household['income']
        self.occupancy_pref = household['occupancy_pref']
        self.room_pref = household['room_pref']
        self.story = []
        self.residence = Residence(env, household)
        
        self.setStory(self.residence)
    
    def setStory(self, residence):
        self.story.append(
        '{0} lives in a {1} bedroom {2} ({3}). '.format(self.name, 
                                                        residence.rooms, 
                                                        residence.occupancy,
                                                        residence.address
                                                        )
        )




In [5]:
def inspect_residence(env, household):  
    
    household.residence.inspect_start = env.now

    inspection_time = 14
    yield env.timeout(inspection_time)

    household.residence.inspection.succeed()
    yield household.residence.inspection
    household.residence.inspection.processed

    household.residence.inspect_stop = env.now

    if household.residence.damage == 0: 
        household.story.append(
            '{0}\'s {1} ({2}) was inspected after {3} days and was green tagged. '.format(household.name, 
                                                    household.residence.occupancy, 
                                                    household.residence.address,
                                                    inspection_time)
            )

        env.exit()

    if household.residence.damage == 1: 
        household.story.append(
            '{0}\'s {1} ({2}) was inspected after {3} days and was yellow tagged. '.format(household.name, 
                                                    household.residence.occupancy, 
                                                    household.residence.address,
                                                    inspection_time)
            )
    if household.residence.damage == 2: 
        household.story.append(
            '{0}\'s {1} ({2}) was inspected after {3} days and was red tagged. '.format(household.name, 
                                                    household.residence.occupancy, 
                                                    household.residence.address,
                                                    inspection_time)
            )



In [6]:
def search_housing(env, household, housing_stock):
    if household.residence.inspection.processed and household.residence.damage == 2:
        search_start = env.now

        household.story.append(
        '{0} started searching for a {1} bedroom {2} that costs less than ${3} per month on day {4}. '.format(household.name, 
                                                               household.room_pref,
                                                               household.occupancy_pref,
                                                                household.income,
                                                                            search_start)
        )

        yield env.timeout(100)



        new_residence = yield housing_stock.get(lambda getResidence:
                                                getResidence.damage == 0 and
                                                getResidence.occupancy == household.occupancy_pref and 
                                                getResidence.rooms >= household.room_pref and
                                                getResidence.cost <= household.income
                                               )

        yield housing_stock.put(household.residence)
        
        household.residence = new_residence





        search_stop = env.now

        household.story.append(
        '{0} occupied a ${1} per month {2} bedroom {3} ({4}) after {5} days. '.format(household.name, 
                                                               new_residence.cost,
                                                                new_residence.rooms, 
                                                               new_residence.occupancy,
                                                                new_residence.address,
                                                                search_stop)
        )
        
        return 'found housing'
    return 'do not need housing'

In [7]:
def household_leave():
    #yield env.timeout(10, "I am leaving!")
    print('I am leaving!')

In [8]:
# # CONDITIONAL SEARCH_HOUSING
# def search_housing(env, household, housing_stock):
#     if household.residence.inspection.processed and household.residence.damage == 2:
#         search_start = env.now

#         household.story.append(
#         '{0} started searching for a {1} bedroom {2} that costs less than ${3} per month on day {4}. '.format(household.name, 
#                                                                household.room_pref,
#                                                                household.occupancy_pref,
#                                                                 household.income,
#                                                                             search_start)
#         )

#         yield env.timeout(10)



#         new_residence = yield housing_stock.get(lambda getResidence:
#                                                 getResidence.damage == 0 and
#                                                 getResidence.occupancy == household.occupancy_pref and 
#                                                 getResidence.rooms >= household.room_pref and
#                                                 getResidence.cost <= household.income
#                                                )

#         yield housing_stock.put(household.residence)
        
#         household.residence = new_residence





#         search_stop = env.now

#         household.story.append(
#         '{0} occupied a ${1} per month {2} bedroom {3} ({4}) after {5} days. '.format(household.name, 
#                                                                new_residence.cost,
#                                                                 new_residence.rooms, 
#                                                                new_residence.occupancy,
#                                                                 new_residence.address,
#                                                                 search_stop)
#         )
        
#         return 'found housing'

In [9]:
# Manually create household input data dictionary for testing
households_inputs_dict = {'name' : ['Bill', 'Boyd', 'Bobby', 'Biff'],
                          'income' : [5000.0, 4000.0, 2000.0, 3000.0],
                        'occupancy_pref' : ['house','condo','apartment','apartment'],
                        'room_pref' : [3, 1, 1, 2],
                          'address' : ['101', '102', '103', '104'],
                          'occupancy' : ['house', 'condo', 'apartment', 'apartment'],
                        'cost' : [4000.0,3000.0,1000.0,2000.0],
                        'rooms' : [3, 1, 1, 2],
                       'floors' : [1, 1, 1, 1],
                       'age' : [1, 1, 1, 1],
                       'value' : [1, 1, 1, 1],
                       'damage' : [0, 1, 1, 2] 
                    }
# Create dataframe from dictionary
households_df = pd.DataFrame(households_inputs_dict)

# Re-order dataframe columns
#households_df = households_df[['name','income','occupancy_pref','room_pref']]
households_df

,address,age,cost,damage,floors,income,name,occupancy,occupancy_pref,room_pref,rooms,value
0,101,1,4000,0,1,5000,Bill,house,house,3,3,1
1,102,1,3000,1,1,4000,Boyd,condo,condo,1,1,1
2,103,1,1000,1,1,2000,Bobby,apartment,apartment,1,1,1
3,104,1,2000,2,1,3000,Biff,apartment,apartment,2,2,1


In [10]:
# Manually create housing input data dictionary for testing
housing_stock_dict = {'address' : ['105', '106', '107', '108'],
                          'occupancy' : ['house', 'condo', 'apartment', 'apartment'],
                        'cost' : [4000.0,3000.0,1000.0,2000.0],
                        'rooms' : [3, 1, 1, 2],
                       'floors' : [1, 1, 1, 1],
                       'age' : [1, 1, 1, 1],
                       'value' : [1, 1, 1, 1],
                       'damage' : [0, 0, 0, 0]      
                    }
# Create dataframe from dictionary
housing_stock_df = pd.DataFrame(housing_stock_dict)

# Re-order dataframe columns
housing_stock_df

,address,age,cost,damage,floors,occupancy,rooms,value
0,105,1,4000,0,1,house,3,1
1,106,1,3000,0,1,condo,1,1
2,107,1,1000,0,1,apartment,1,1
3,108,1,2000,0,1,apartment,2,1


In [11]:
env = simpy.Environment()

In [12]:
#Instantiate housing stock (not assigned to any households)
housing_stock = simpy.FilterStore(env)

for i in housing_stock_df.index:
    housing_stock.put(Residence(env, housing_stock_df.loc[i]))


In [13]:
#Instantiate household objects
households = {}
for i in households_df.index:
    households[i] = Household(env, households_df.loc[i])

In [14]:
def master_process(env, household, housing_stock):
    for household in households.keys():
        yield env.process(inspect_residence(env, households[household]))
        
        patience = env.timeout(3, value='give up') 
        housing_search = env.process(search_housing(env, households[household], housing_stock))
        search_decision = yield patience | housing_search
         
        
        if search_decision == {patience: 'give up'}:
            print(search_decision[patience])
        if search_decision == {housing_search: 'found housing'}:
            print(search_decision[housing_search])
            print(housing_search)
        if search_decision == {housing_search: 'do not need housing'}:
            print(search_decision[housing_search])
            print(housing_search)
                      
                      

In [15]:
# for household in households.keys():
#     env.process(inspect_residence(env, households[household]))
#     env.process(search_housing(env, households[household], housing_stock))

#     dh = env.process(damage_housing(env, households[household]))
#     sh = env.process(search_housing(env, households[household]))
    

#     start_delayed(env, search_housing(env, households[household], housing_stock), 2)
    



In [16]:
env.process(master_process(env, households, housing_stock))

<Process(master_process) object at 0x10931f438>

In [17]:
env.run(until=1000)

do not need housing
<Process(search_housing) object at 0x1091a7cc0>
do not need housing
<Process(search_housing) object at 0x1091a74a8>
do not need housing
<Process(search_housing) object at 0x10931f470>
give up


In [18]:
households[0].residence.inspection.processed

True

In [19]:
for household in households.keys():
    print(''.join(households[household].story))
    print()

Bill lives in a 3 bedroom house (101). Bill's house (101) was inspected after 14 days and was green tagged. 

Boyd lives in a 1 bedroom condo (102). Boyd's condo (102) was inspected after 14 days and was yellow tagged. 

Bobby lives in a 1 bedroom apartment (103). Bobby's apartment (103) was inspected after 14 days and was yellow tagged. 

Biff lives in a 2 bedroom apartment (104). Biff's apartment (104) was inspected after 14 days and was red tagged. Biff started searching for a 2 bedroom apartment that costs less than $3000.0 per month on day 56. Biff occupied a $2000.0 per month 2 bedroom apartment (108) after 156 days. 



In [20]:
print("There are {0} households still waiting for housing".format(len(housing_stock.get_queue)))

There are 0 households still waiting for housing


In [21]:
housing_stock.items[3].address

'104'